# School Timetable Functions

**Purpose:** This notebook contains all functions required to build, solve, and format a school timetable using OR-Tools CP-SAT solver.

- `Profiler` -> Profiles execution time and memory usage during key operations.
- `load_data()` -> Load CSV files into DataFrames.
- `build_model()` -> Define CP-SAT variables, constraints, and objective.
- `solve_model()` -> Solve the model and extract the timetable.
- `format_output()` -> Format the solution and merge with timeslot information.

## Profiling Utilities

- The `Profiler` class is a **context manager** for tracking:
  - **Execution time** of code blocks.
  - **Memory usage** (current and peak) using `tracemalloc`.
- It helps evaluate performance and optimize model-building and solving steps.

In [ ]:
# Profiling utilities
class Profiler:
    """Simple context manager for timing and memory profiling."""

    def __init__(self, label: str):
        self.label = label
        self.start_time = None

    def __enter__(self):
        logging.info(f"Starting: {self.label}")
        self.start_time = time.perf_counter()
        tracemalloc.start()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        elapsed = time.perf_counter() - self.start_time
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        logging.info(
            f"Finished: {self.label} | Time: {elapsed:.2f}s | "
            f"Memory: {current/1024:.2f} KB | Peak: {peak/1024:.2f} KB"
        )

## Data Loading Function

- Loads all necessary CSV files:
  - `class_list.csv`, `timeslots.csv`, `lessons_required.csv`, `weekdays.csv`, `teacher_classes.csv`, `student_classes.csv`, `teacher_list.csv`.
- Drops any excluded classes (e.g., "isizulu-12").
- Returns all datasets as pandas DataFrames.

In [ ]:
# Data loading
def load_data() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Load all required CSV files into DataFrames."""
    class_list = pd.read_csv("class_list.csv")
    timeslots_days = pd.read_csv("timeslots.csv")
    lessons_required = pd.read_csv("lessons_required.csv")
    weekday = pd.read_csv("weekdays.csv")
    teacher_classes = pd.read_csv("teacher_classes.csv")
    student_classes = pd.read_csv("student_classes.csv")
    teacher_list = pd.read_csv("teacher_list.csv")

    # Drop excluded classes (hardcoded rule)
    student_classes = student_classes[student_classes["class"] != "isizulu-12"]
    teacher_classes = teacher_classes[teacher_classes["class"] != "isizulu-12"]

    return (
        class_list, timeslots_days, lessons_required,
        weekday, teacher_classes, student_classes, teacher_list
    )


## Model Builder

- Creates a CP-SAT model and binary decision variables:
  - `x[(class, day, period)] = 1` if class is scheduled at that time.
- **Constraints**:
  1. Each class must have the required number of lessons.
  2. Teachers cannot be double-booked in the same period.
  3. Students cannot attend multiple classes at the same time.
  4. Soft constraints:
     - Spread lessons evenly across days.
     - Consecutive lessons for classes when possible.
- **Objective**: Minimize violations (student clashes + uneven lesson distribution).

In [ ]:
# Model builder
def build_model(
    classes: List[str],
    days: List[str],
    periods: range,
    num_lessons: Dict[str, int],
    class_teacher: Dict[str, str],
    class_students: Dict[str, List[str]]
) -> Tuple[cp_model.CpModel, Dict[Tuple[str, str, int], cp_model.IntVar]]:
    """Build and return the CP-SAT model and decision variables."""

    model = cp_model.CpModel()

    # Variables: binary for each (class, day, period)
    x = {}
    for c in classes:
        for d in days:
            for p in periods:
                x[(c,d,p)] = model.NewBoolVar(f'x_{c}_{d}_{p}')

    # Constraint: required lessons
    for c in classes:
        model.Add(sum(x[(c, d, p)] for d in days for p in periods) == num_lessons.get(c, 0))

    # Constraint: teacher cannot be double-booked

    for d in days:
        for p in periods:
            for t in set(class_teacher.values()): 
                teaching_classes = [cls for cls, teacher in class_teacher.items() if teacher == t]
                if teaching_classes:
                    model.Add(sum(x[(c,d,p)] for c in teaching_classes if c in classes) <= 1)

    # Constraint: students cannot be double-booked
    violations = []
    for d in days:
        for p in periods:
            all_students = {s for students in class_students.values() for s in students}
            for s in all_students:
                enrolled_classes = [c for c, students in class_students.items() if s in students]
                if enrolled_classes:
                    v = model.NewBoolVar(f"violation[{s},{d},{p}]")
                    violations.append(v)
                    model.Add(sum(x[(c,d,p)] for c in enrolled_classes if c in classes) <= 1 + v)

    # Soft constraint: spread lessons across days
    violations_spread = []
    for c in classes:
        lessons_needed = num_lessons[c]
        avg = lessons_needed / len(days)
        for d in days:
            day_lessons = sum(x[(c, d, p)] for p in periods)
            dev = model.NewIntVar(0, lessons_needed, f"spread_dev[{c},{d}]")
            violations_spread.append(dev)
            model.Add(day_lessons - math.ceil(avg) <= dev)
            model.Add(math.floor(avg) - day_lessons <= dev)

    # Soft/Hard constraint: if there are multiple lessons of the same class, they should be consecutive if possible
    for c in classes:
        for d in days:
            vars_day = [x[(c, d, p)] for p in periods]
            is_two = model.NewBoolVar(f"{c}_{d}_is_two")
            model.Add(sum(vars_day) == 2).OnlyEnforceIf(is_two)
            model.Add(sum(vars_day) <= 1).OnlyEnforceIf(is_two.Not()) 
            if len(periods) >= 2:
                pair_bools = []
                for p_idx in range(len(periods) - 1):
                    pair = model.NewBoolVar(f"{c}_{d}_pair_{p_idx}")
                    pair_bools.append(pair)
                    # pair -> both vars are 1
                    model.Add(vars_day[p_idx] == 1).OnlyEnforceIf(pair)
                    model.Add(vars_day[p_idx+1] == 1).OnlyEnforceIf(pair)
                model.Add(sum(pair_bools) >= 1).OnlyEnforceIf(is_two)
    
    # Objective: minimize clashes + uneven spread
    model.Minimize(sum(violations) + sum(violations_spread))

    return model, x

## Solver Function

- Solves the CP-SAT model using `CpSolver`.
- Limits maximum solving time to 20 seconds.
- Extracts decision variables where class is scheduled.
- Returns solution as a pandas DataFrame with columns `weekday`, `period`, `class`.

In [ ]:
# Solver
def solve_model(model: cp_model.CpModel, x: Dict) -> pd.DataFrame:
    """Solve the CP-SAT model and return the solution DataFrame."""
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 20  # time cap

    status = solver.Solve(model)
    if status not in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        raise RuntimeError("No feasible solution found.")

    rows = [(d, p, c) for (c, d, p), var in x.items() if solver.Value(var) == 1]
    return pd.DataFrame(rows, columns=["weekday", "period", "class"])

## Output Formatting Function

- Merges solver output with timeslot information from `timeslots.csv`.
- Converts timeslot strings to structured start times.
- Returns final timetable with columns `timeslot`, `class`.

In [ ]:
# Post-processing
def format_output(output: pd.DataFrame, timeslots_days: pd.DataFrame) -> pd.DataFrame:
    """Merge solver output with timeslot info and produce final timetable."""
    # Parse timeslot string
    period_interval = timeslots_days.copy()
    split_cols = period_interval["timeslot"].str.rsplit(" - ", n=1, expand=True)

    period_interval["start_time"] = (
        split_cols[0]
        .str.replace(r"^[A-Z]+\s*", "", regex=True)
        .str.replace("_", ":", regex=False)
        .str.strip()
    )

    # Map periods to start times
    unique_times = period_interval["start_time"].drop_duplicates().tolist()
    period_start_times = pd.DataFrame({
        "period": range(len(unique_times)),
        "start_time": unique_times
    })

    period_interval = period_interval.merge(period_start_times, on="start_time", how="left")
    return output.merge(period_interval, on=["period", "weekday"], how="left")[["timeslot", "class"]]